In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json

In [198]:
#set the keys
yelp_id = os.environ["YELP_FUSION_API"]

In [3]:
class yelp_api_request:
    """
    Get a list of parameters to construct api request
    """
    def __init__(self, api_key, search_path, term, location, search_limit, latitude, longitude):
        self.api_key = api_key
        self.api_host = 'https://api.yelp.com'
        self.search_path = search_path
        self.term = term
        self.location = location
        self.search_limit = search_limit
        self.latitude = latitude
        self.longitude = longitude

    def request(self, url_params=None):
        """Given your API_KEY, send a GET request to the API.
        Args:
            host (str): The domain host of the API.
            path (str): The path of the API after the domain.
            API_KEY (str): Your API Key.
            url_params (dict): An optional set of query parameters in the request.
        Returns:
            dict: The JSON response from the request.
        """
        url = '{0}{1}'.format(self.api_host, self.search_path)
        headers = {
            'Authorization': "Bearer %s" % self.api_key,
        }
        response = requests.request('GET', url, headers=headers, params=url_params)
        return response.json()

    def search(self):
        """Query the Search API by a search term and location.
        Args:
            term (str): The search term passed to the API.
            location (str): The search location passed to the API.

        Returns:
            dict: The JSON response from the request.
        """
        url_params = {
            'term': self.term.replace(' ', '+'),
            # 'location': self.location.replace(' ', '+'),
            'limit': self.search_limit,
            'latitude': self.latitude,
            'longitude': self.longitude,
            'radius': 1000,
            'text': 'bike'
        }
        return self.request(url_params=url_params)

In [145]:
API_KEY = os.environ["YELP_FUSION_API"]

SEARCH_PATH = '/v3/businesses/search'
# SEARCH_PATH = '/v3/autocomplete'

# Defaults for our simple example.
DEFAULT_TERM = 'park'
DEFAULT_LOCATION = 'Toronto, CA'
LATITUDE = "43.665269"
LONGITUDE = "-79.319796"
SEARCH_LIMIT = 20

response = yelp_api_request(API_KEY, SEARCH_PATH, DEFAULT_TERM, DEFAULT_LOCATION, SEARCH_LIMIT, LATITUDE, LONGITUDE).search()
response



{'error': {'code': 'ACCESS_LIMIT_REACHED',
  'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}

In [200]:
# Get all the information from the bikes file
bike_dataframe = pd.read_csv('bikes.csv')
bike_dataframe


In [1]:
for i in response:
    print(i)
response['businesses'][0]['categories'][0]

# print(response.keys())

park_dataframe = []

category_list = []
index_list = []
for i in response['businesses']:
    index_list.append(bike_dataframe['index'][0])
    category_list.append(i['categories'][-1:][0]['title'])
park_db = pd.DataFrame()
park_db['citybike_index'] = index_list
park_db_interm = pd.json_normalize(response['businesses'])
park_db[list(park_db_interm.columns.values)] = park_db_interm
park_db.drop(columns=['categories'])
park_db['categories'] = category_list
park_db.iloc[:, 0:14]
park_dataframe.append(park_db.iloc[:, 0:14])
park_dataframe

In [2]:
API_KEY = os.environ["YELP_FUSION_API"]
SEARCH_PATH = '/v3/businesses/search'
DEFAULT_TERM = 'park'
DEFAULT_LOCATION = 'Toronto, CA'
SEARCH_LIMIT = 10

park_dataframe = []
for i in bike_dataframe.index:
    
    LATITUDE = bike_dataframe['latitude'][i]
    LONGITUDE = bike_dataframe['longitude'][i]
    response = yelp_api_request(API_KEY, SEARCH_PATH, DEFAULT_TERM, DEFAULT_LOCATION, SEARCH_LIMIT, LATITUDE, LONGITUDE).search()
    
    category_list = []
    index_list = []
    
    for j in response['businesses']:
        index_list.append(bike_dataframe['index'][i])
        # category_list.append(i['categories'][-1:][0]['title'])
    park_db = pd.DataFrame()
    park_db['citybike_index'] = index_list
    park_db_interm = pd.json_normalize(response['businesses'])
    park_db[list(park_db_interm.columns.values)] = park_db_interm
    # park_db.drop(columns=['categories'])
    # park_db['categories'] = category_list
    park_dataframe.append(park_db.iloc[:, 0:14])

In [148]:
# Below, a backup of the dataframe is created and the data gathered is parsed through, with the necessary data being stored in 'yelp_park_data.csv'.
park_dataframe
park_dataframe2 = park_dataframe

In [3]:
# park_dataframe2
park_dataframe3 = pd.DataFrame(park_dataframe2)
# park_dataframe3.to_csv('yelp_park_df.csv')

In [162]:
park_dataframe2[0]

,citybike_index,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,phone,display_phone,distance
0,0,Drn62oroI35wMeg1aMWlCw,woodbine-park-toronto,Woodbine Park,https://s3-media2.fl.yelpcdn.com/bphoto/RPuAR4...,False,https://www.yelp.com/biz/woodbine-park-toronto...,10,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,[],,,594.427716
1,0,BH-dRhebecIxScLPZh9-FA,ashbridge-estate-toronto,Ashbridge Estate,https://s3-media2.fl.yelpcdn.com/bphoto/bd_Tc3...,False,https://www.yelp.com/biz/ashbridge-estate-toro...,1,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,[],+14163255000,+1 416-325-5000,237.321078
2,0,aTPPgfGzmemaKZdpkymt8A,greenwood-park-toronto,Greenwood Park,https://s3-media3.fl.yelpcdn.com/bphoto/PfBfpt...,False,https://www.yelp.com/biz/greenwood-park-toront...,11,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.0,[],,,814.321414
3,0,Tfb0k0UCxMhOe-YvspX3Pg,woodbine-beach-toronto,Woodbine Beach,https://s3-media2.fl.yelpcdn.com/bphoto/q9qGH8...,False,https://www.yelp.com/biz/woodbine-beach-toront...,26,"[{'alias': 'beaches', 'title': 'Beaches'}]",4.5,[],,,943.134995
4,0,-Za5mjo-CYYUMsd1r8GC7Q,ashbridges-bay-park-toronto,Ashbridges Bay Park,https://s3-media3.fl.yelpcdn.com/bphoto/zhjfNf...,False,https://www.yelp.com/biz/ashbridges-bay-park-t...,30,"[{'alias': 'beaches', 'title': 'Beaches'}, {'a...",3.5,[],,,959.632364
5,0,2xpzryM9Tio1SJin5l0Hmg,jonathan-ashbridge-park-toronto,Jonathan Ashbridge Park,https://s3-media4.fl.yelpcdn.com/bphoto/tytN2r...,False,https://www.yelp.com/biz/jonathan-ashbridge-pa...,1,"[{'alias': 'parks', 'title': 'Parks'}]",2.0,[],,,73.308008
6,0,dr57Gr2FeEGL7xSutbGSaQ,measurement-park-toronto,Measurement Park,https://s3-media2.fl.yelpcdn.com/bphoto/BKZYC0...,False,https://www.yelp.com/biz/measurement-park-toro...,3,"[{'alias': 'parks', 'title': 'Parks'}]",3.0,[],,,595.454107
7,0,-YiW2V4hGyAHx5k6R2610Q,ashbridges-bay-skate-park-toronto,Ashbridges Bay Skate Park,https://s3-media2.fl.yelpcdn.com/bphoto/5iUk8n...,False,https://www.yelp.com/biz/ashbridges-bay-skate-...,2,"[{'alias': 'skate_parks', 'title': 'Skate Park...",3.5,[],,,393.911448


In [163]:
park_dataframe2[1]

,citybike_index,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,phone,display_phone,distance
0,1,JODWOfeEICAlO2T_VAEbSg,earlscourt-park-toronto,Earlscourt Park,https://s3-media4.fl.yelpcdn.com/bphoto/T7TgPz...,False,https://www.yelp.com/biz/earlscourt-park-toron...,7,"[{'alias': 'dog_parks', 'title': 'Dog Parks'}]",3.5,[],,,500.794541
1,1,l_CGwZemu7WGp8dbsdrfzQ,wallace-emerson-community-centre-toronto,Wallace Emerson Community Centre,https://s3-media2.fl.yelpcdn.com/bphoto/Uy1bGb...,False,https://www.yelp.com/biz/wallace-emerson-commu...,6,"[{'alias': 'swimmingpools', 'title': 'Swimming...",3.0,[],+14163920039,+1 416-392-0039,686.731559
2,1,dzWdAHIEQPlAt_kbiuohgg,macgregor-park-toronto,MacGregor Park,https://s3-media4.fl.yelpcdn.com/bphoto/9R_LNQ...,False,https://www.yelp.com/biz/macgregor-park-toront...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,[],,,733.065260
3,1,SucEHgmWcUrhs9z8ZJ85pw,symington-avenue-playground-toronto,Symington Avenue Playground,https://s3-media3.fl.yelpcdn.com/bphoto/xXsMP8...,False,https://www.yelp.com/biz/symington-avenue-play...,1,"[{'alias': 'playgrounds', 'title': 'Playground...",3.0,[],,,715.641897


In [174]:
final_df = pd.DataFrame()
# for i in park_dataframe2:
#     finapd.concat([final_df, i], ignore_index=True)
final_df = pd.concat(park_dataframe2).reset_index(drop=True)
final_df.head()

,citybike_index,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,phone,display_phone,distance
0,0.0,Drn62oroI35wMeg1aMWlCw,woodbine-park-toronto,Woodbine Park,https://s3-media2.fl.yelpcdn.com/bphoto/RPuAR4...,False,https://www.yelp.com/biz/woodbine-park-toronto...,10.0,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,[],,,594.427716
1,0.0,BH-dRhebecIxScLPZh9-FA,ashbridge-estate-toronto,Ashbridge Estate,https://s3-media2.fl.yelpcdn.com/bphoto/bd_Tc3...,False,https://www.yelp.com/biz/ashbridge-estate-toro...,1.0,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,[],+14163255000,+1 416-325-5000,237.321078
2,0.0,aTPPgfGzmemaKZdpkymt8A,greenwood-park-toronto,Greenwood Park,https://s3-media3.fl.yelpcdn.com/bphoto/PfBfpt...,False,https://www.yelp.com/biz/greenwood-park-toront...,11.0,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.0,[],,,814.321414
3,0.0,Tfb0k0UCxMhOe-YvspX3Pg,woodbine-beach-toronto,Woodbine Beach,https://s3-media2.fl.yelpcdn.com/bphoto/q9qGH8...,False,https://www.yelp.com/biz/woodbine-beach-toront...,26.0,"[{'alias': 'beaches', 'title': 'Beaches'}]",4.5,[],,,943.134995
4,0.0,-Za5mjo-CYYUMsd1r8GC7Q,ashbridges-bay-park-toronto,Ashbridges Bay Park,https://s3-media3.fl.yelpcdn.com/bphoto/zhjfNf...,False,https://www.yelp.com/biz/ashbridges-bay-park-t...,30.0,"[{'alias': 'beaches', 'title': 'Beaches'}, {'a...",3.5,[],,,959.632364


In [176]:
final_df.to_csv('yelp_park_data.csv')

In [183]:
categories_df = final_df['categories']

In [197]:
# put the category names into a column.
yelp_park_data_categories = []
for i in categories_df:
    yelp_park_data_categories.append(i[0]['alias'])

final_df2['categories_names'] = yelp_park_data_categories

final_df2.to_csv('yelp_park_data.csv')

In [2]:
# The raw data was put into the file 'yelp_park_data.csv'
categories_df = pd.read_csv('yelp_park_data.csv')
categories_df = categories_df.drop(categories_df.columns[0], axis=1)

In [3]:
categories_df

,citybike_index,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,phone,display_phone,distance,categories_names
0,0.0,Drn62oroI35wMeg1aMWlCw,woodbine-park-toronto,Woodbine Park,https://s3-media2.fl.yelpcdn.com/bphoto/RPuAR4...,False,https://www.yelp.com/biz/woodbine-park-toronto...,10.0,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,[],NaN,NaN,594.427716,parks
1,0.0,BH-dRhebecIxScLPZh9-FA,ashbridge-estate-toronto,Ashbridge Estate,https://s3-media2.fl.yelpcdn.com/bphoto/bd_Tc3...,False,https://www.yelp.com/biz/ashbridge-estate-toro...,1.0,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,[],1.416326e+10,+1 416-325-5000,237.321078,parks
2,0.0,aTPPgfGzmemaKZdpkymt8A,greenwood-park-toronto,Greenwood Park,https://s3-media3.fl.yelpcdn.com/bphoto/PfBfpt...,False,https://www.yelp.com/biz/greenwood-park-toront...,11.0,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.0,[],NaN,NaN,814.321414,parks
3,0.0,Tfb0k0UCxMhOe-YvspX3Pg,woodbine-beach-toronto,Woodbine Beach,https://s3-media2.fl.yelpcdn.com/bphoto/q9qGH8...,False,https://www.yelp.com/biz/woodbine-beach-toront...,26.0,"[{'alias': 'beaches', 'title': 'Beaches'}]",4.5,[],NaN,NaN,943.134995,beaches
4,0.0,-Za5mjo-CYYUMsd1r8GC7Q,ashbridges-bay-park-toronto,Ashbridges Bay Park,https://s3-media3.fl.yelpcdn.com/bphoto/zhjfNf...,False,https://www.yelp.com/biz/ashbridges-bay-park-t...,30.0,"[{'alias': 'beaches', 'title': 'Beaches'}, {'a...",3.5,[],NaN,NaN,959.632364,beaches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4749,490.0,7whRTbwmDUq213aWE5UB9w,massey-harris-park-toronto,Massey Harris Park,https://s3-media4.fl.yelpcdn.com/bphoto/geRhkO...,False,https://www.yelp.com/biz/massey-harris-park-to...,1.0,"[{'alias': 'parks', 'title': 'Parks'}]",3.0,[],NaN,NaN,631.899173,parks
4750,490.0,91aTiPAUqL0IOoaWLS2PSQ,gateway-park-toronto,Gateway Park,https://s3-media2.fl.yelpcdn.com/bphoto/p9bC_s...,False,https://www.yelp.com/biz/gateway-park-toronto?...,1.0,"[{'alias': 'parks', 'title': 'Parks'}]",3.0,[],1.416338e+10,+1 416-338-4386,635.585840,parks
4751,491.0,A1Ep3RYtCLFeEOg2pAiUCw,taylor-creek-park-toronto,Taylor Creek Park,https://s3-media2.fl.yelpcdn.com/bphoto/Sfhb8T...,False,https://www.yelp.com/biz/taylor-creek-park-tor...,11.0,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,[],NaN,NaN,744.453991,parks
4752,491.0,ceP48LDxQCYMnDu9hOwXqw,e-t-seton-park-toronto,E.T. Seton Park,https://s3-media4.fl.yelpcdn.com/bphoto/j2owAJ...,False,https://www.yelp.com/biz/e-t-seton-park-toront...,1.0,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,[],NaN,NaN,643.642675,parks
